# Coarse-grain a simple transition matrix

To illustrate the application of *pyGPCCA* we will coarse-grain a simple transition matrix *P* calculated from a toy adjacency matrix *W* found in *[S. Roeblitz and M. Weber, Fuzzy spectral clustering by PCCA+: application to Markov state models and data classification. Adv Data Anal Classif 7, 147-179 (2013)](https://doi.org/10.1007/s11634-013-0134-6)*. 

Firstly, we will import needed packages like ``numpy`` and of course ``pygpcca``:

In [1]:
import numpy as np
import pygpcca as gp

Next, we define the aforementioned adjacency matrix *W* and calculate a row-stochastic (meaning that the rows of *P* each sum up to one) transition matrix *P* from it:

In [2]:
# Choose zero pertubation mu of the adjacency matrix.
mu = 0
W = np.array(
        [
            [1000, 100, 100, 10, 0, 0, 0, 0, 0],
            [100, 1000, 100, 0, 0, 0, 0, 0, 0],
            [100, 100, 1000, 0, mu, 0, 0, 0, 0],
            [10, 0, 0, 1000, 100, 100, 10, 0, 0],
            [0, 0, mu, 100, 1000, 100, 0, 0, 0],
            [0, 0, 0, 100, 100, 1000, 0, mu, 0],
            [0, 0, 0, 10, 0, 0, 1000, 100, 100],
            [0, 0, 0, 0, 0, mu, 100, 1000, 100],
            [0, 0, 0, 0, 0, 0, 100, 100, 1000],
        ],
        dtype=np.float64,
    )
# Only make non-zero rows stochastic, otherwise we might divide by zero later.
# (This is just a general precaution and not explicitly necessary in the special case considered here.)
row = np.sum(W, axis=1) > 0.0001
P = W.copy()
W_ = W[row, :]
# Calculate the transition matrix from W.
P[row, :] = np.diag(1.0 / np.sum(W_, axis=1)) @ W_
P

array([[0.82644628, 0.08264463, 0.08264463, 0.00826446, 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.08333333, 0.83333333, 0.08333333, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.08333333, 0.08333333, 0.83333333, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.00819672, 0.        , 0.        , 0.81967213, 0.08196721,
        0.08196721, 0.00819672, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.08333333, 0.83333333,
        0.08333333, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.08333333, 0.08333333,
        0.83333333, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.00826446, 0.        ,
        0.        , 0.82644628, 0.08264463, 0.08264463],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.08333333, 0.83333333, 0.08333333],


Following this, we initialize a *GPCCA* object from the transition matrix *P*:

In [3]:
gpcca = gp.GPCCA(P)

Afterwards, we can get a list of *minChi* values for numbers of macrostates *m* in an interval *\[2, 8\]* to determine an interval
*\[m_min, m_max\]* of (nearly) optimal numbers of macrostates for clustering:

In [5]:
gpcca.minChi(2, 8)

[-2.220446049250313e-16,
 -3.4545621951634044e-16,
 -0.8336674986613027,
 -0.9933333388201755,
 -0.9081679698248197,
 -0.4168337493306544,
 -0.24719203433643516]

This might result in a warning:

```
UserWarning: The Schur vectors aren't D-orthogonal so they are D-orthogonalized.
warnings.warn("The Schur vectors aren't D-orthogonal so they are D-orthogonalized.")
```
  
This is uncritical and has merely technical reasons.

The *minChi* citerion states that cluster numbers *m* (i.e. clustering into *m* clusters) with a *minChi* value close to zero will potentially result in a optimal (meaning especially *crisp* or sharp) clustering.
Obviously, only *m*=3 qualifies as non-trivially (potentially) optimal, since *m*=2 is always (trivially) optimal.

Now, we would typically optimize the clustering for numbers of macrostates *m* in the previously determined interval *\[m_min, m_max\]* and find the optimal number of macrostates *n_metastable* in the given interval. Since this interval would here actually only span one number, i.e. \[3\], we will optimize the clustering for the whole interval of possible cluster numbers to see what happens: 

In [6]:
gpcca.optimize({'m_min':2, 'm_max':8})

The optimized *GPCCA* object is returned above and we can now access different properties of it.

The optimal number of macrostates *n_metastable* can be accessed via:

In [7]:
gpcca.n_metastable

3

The optimal number of clusters or macrostates is *n_metastable*=3 as expected.

The optimal coarse-grained matrix can be accessed via:

In [8]:
gpcca.coarse_grained_transition_matrix

array([[9.94827531e-01, 2.58623439e-03, 2.58623439e-03],
       [2.59294034e-03, 9.97348758e-01, 5.83020119e-05],
       [2.59294034e-03, 5.83020119e-05, 9.97348758e-01]])

The memberships are available via:

In [9]:
gpcca.memberships

array([[3.11152841e-02, 6.99624142e-04, 9.68185092e-01],
       [4.65713093e-16, 5.76285949e-16, 1.00000000e+00],
       [1.61136004e-16, 6.92952765e-18, 1.00000000e+00],
       [9.37930375e-01, 3.10348127e-02, 3.10348127e-02],
       [1.00000000e+00, 7.80038068e-18, 4.55528202e-16],
       [1.00000000e+00, 1.08953359e-15, 0.00000000e+00],
       [3.11152841e-02, 9.68185092e-01, 6.99624142e-04],
       [0.00000000e+00, 1.00000000e+00, 3.41183190e-16],
       [4.47211372e-16, 1.00000000e+00, 5.01328432e-16]])

There are many more properties that can be accessed as you can see in the API documentation <a href="https://pygpcca.readthedocs.io/en/latest/api/pygpcca.GPCCA.html" target="_blank">here</a>.